In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [37]:
import numpy as np
import re

from tensorflow import keras
from tensorflow.keras.layers import Dense, SimpleRNN, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from matplotlib import pyplot as plt

In [95]:
with open('pogovorki-plus.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    text = text.replace('\ufeff','') #убираем первый невидимый символ
    text = re.sub(r'[^А-я ]', '', text) #заменяем все символы кроме кириллицы на пустые символы

In [98]:
#парсим текст как последовательность символов
num_characters=34 #33 буквы + пробел
tokenizer = Tokenizer(num_words=num_characters, char_level=True) #Токенизируем на уровне символов
tokenizer.fit_on_texts([text])
print(tokenizer.word_index)

{' ': 1, 'о': 2, 'е': 3, 'а': 4, 'т': 5, 'н': 6, 'и': 7, 'р': 8, 'в': 9, 'с': 10, 'д': 11, 'л': 12, 'к': 13, 'у': 14, 'м': 15, 'ь': 16, 'п': 17, 'г': 18, 'б': 19, 'я': 20, 'з': 21, 'ы': 22, 'ч': 23, 'й': 24, 'ш': 25, 'ж': 26, 'х': 27, 'ю': 28, 'ц': 29, 'щ': 30, 'ф': 31, 'ъ': 32, 'э': 33}


In [99]:
inp_chars = 6
data = tokenizer.texts_to_matrix(text) #преобразуем исходных текст в массив OHE
n = data.shape[0] - inp_chars #так мы предсказываем по трем символам - четвертый

X = np.array([data[i:i + inp_chars, :] for i in range(n)])
Y = data[inp_chars:] #предсказание следующего символа

print(data.shape)

(22370, 34)


In [101]:
model = Sequential()
model.add(Input((inp_chars, num_characters)))
model.add(SimpleRNN(256, activation='tanh'))
model.add(Dense(num_characters, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

history = model.fit(X, Y, batch_size=32, epochs=50)
model.save("model_charsplus_256_50.h5")

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_13 (SimpleRNN)   (None, 256)               74496     
                                                                 
 dense_13 (Dense)            (None, 34)                8738      
                                                                 
Total params: 83,234
Trainable params: 83,234
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
699/699 [==============================] - 3s 4ms/step - loss: 2.7488 - accuracy: 0.2352
Epoch 2/50
699/699 [==============================] - 2s 4ms/step - loss: 2.4743 - accuracy: 0.2869
Epoch 3/50
699/699 [==============================] - 2s 4ms/step - loss: 2.4066 - accuracy: 0.2991
Epoch 4/50
699/699 [==============================] - 3s 4ms/step - loss: 2.3811 - accuracy: 0.3004
Epoch 5/50
699/699 [=========================

In [102]:
model = keras.models.load_model("model_charsplus_256_50.h5")

In [103]:
def buildPhrase(inp_str, str_len=50):
    for i in range(str_len):
        x=[]
        for j in range(i, i + inp_chars):
            x.append(tokenizer.texts_to_matrix(inp_str[j]))
            
        x = np.array(x)
        inp = x.reshape(1,inp_chars, num_characters)
        
        pred = model.predict(inp)
        d = tokenizer.index_word[pred.argmax(axis=1)[0]]
        
        inp_str += d
    return inp_str

In [106]:
res = buildPhrase("солнце")

1/1 [==============================] - 0s 19ms/step


In [107]:
res

'солнце вслуди вода и будет водукизь в тослая скучет захв'

In [66]:
inp_chars = 3
data = tokenizer.texts_to_matrix(text) #преобразуем исходных текст в массив OHE
n = data.shape[0] - inp_chars #так мы предсказываем по трем символам - четвертый

X = np.array([data[i:i + inp_chars, :] for i in range(n)])
Y = data[inp_chars:] #предсказание следующего символа

print(data.shape)

(12499, 34)


In [78]:
model = Sequential()
model.add(Input((inp_chars, num_characters)))
model.add(SimpleRNN(256, activation='tanh'))
model.add(Dense(num_characters, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

history = model.fit(X, Y, batch_size=32, epochs=100)
model.save("model_chars_3_256_100.h5")

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_8 (SimpleRNN)    (None, 256)               74496     
                                                                 
 dense_8 (Dense)             (None, 34)                8738      
                                                                 
Total params: 83,234
Trainable params: 83,234
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
391/391 [==============================] - 2s 2ms/step - loss: 2.7752 - accuracy: 0.2234
Epoch 2/100
391/391 [==============================] - 1s 2ms/step - loss: 2.5073 - accuracy: 0.2787
Epoch 3/100
391/391 [==============================] - 1s 2ms/step - loss: 2.4332 - accuracy: 0.2921
Epoch 4/100
391/391 [==============================] - 1s 2ms/step - loss: 2.3988 - accuracy: 0.2964
Epoch 5/100
391/391 [=====================

391/391 [==============================] - 1s 2ms/step - loss: 1.6395 - accuracy: 0.4722
Epoch 75/100
391/391 [==============================] - 1s 2ms/step - loss: 1.6332 - accuracy: 0.4698
Epoch 76/100
391/391 [==============================] - 1s 2ms/step - loss: 1.6324 - accuracy: 0.4680
Epoch 77/100
391/391 [==============================] - 1s 2ms/step - loss: 1.6224 - accuracy: 0.4717
Epoch 78/100
391/391 [==============================] - 1s 2ms/step - loss: 1.6164 - accuracy: 0.4707
Epoch 79/100
391/391 [==============================] - 1s 2ms/step - loss: 1.6112 - accuracy: 0.4748
Epoch 80/100
391/391 [==============================] - 1s 2ms/step - loss: 1.6043 - accuracy: 0.4766
Epoch 81/100
391/391 [==============================] - 1s 2ms/step - loss: 1.5974 - accuracy: 0.4765
Epoch 82/100
391/391 [==============================] - 1s 2ms/step - loss: 1.5951 - accuracy: 0.4765
Epoch 83/100
391/391 [==============================] - 1s 2ms/step - loss: 1.5931 - accuracy: 

In [79]:
model = keras.models.load_model("model_chars_3_256_100.h5")

In [84]:
res = buildPhrase("сол")

1/1 [==============================] - 0s 14ms/step


In [85]:
res

'солнце словек без хорошадь не и конь горошадь не и ко'